In [17]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
from PIL import Image

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1):
        batch = mnist.train.next_batch(1)
        image, lable = sess.run([tf.reshape(tf.cast(batch[0],tf.float32),[-1,28,28,1]),tf.cast(batch[1],tf.float32)])
        print(image)
        print("==========================")    
        print(lable)
        image_view = sess.run(tf.reshape(tf.cast(batch[0],tf.float32),[28,28]))
        Image.fromarray(image_view).show()   
"""

'\nwith tf.Session() as sess:\n    sess.run(tf.global_variables_initializer())\n    for i in range(1):\n        batch = mnist.train.next_batch(1)\n        image, lable = sess.run([tf.reshape(tf.cast(batch[0],tf.float32),[-1,28,28,1]),tf.cast(batch[1],tf.float32)])\n        print(image)\n        print("==========================")    \n        print(lable)\n        image_view = sess.run(tf.reshape(tf.cast(batch[0],tf.float32),[28,28]))\n        Image.fromarray(image_view).show()   \n'

In [18]:
import tensorflow as tf
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None,10])
x_image = tf.reshape(x,[-1,28,28,1])
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())
#    batch = mnist.train.next_batch(80000)
#    print( sess.run(tf.shape(batch[0])))

In [19]:
# Weight 설정 정규분포 표준편차가 0.1 이상되는 데이터는 제외되고 다시 선택된다.
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# bias 는 0.1로 고정 
def bias_variable(shape):
    inital = tf.constant(0.1, shape=shape)
    return tf.Variable(inital)

#컴플루션 
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

#pooiing
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



In [20]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

In [21]:
# 1. CONV
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [22]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

# 2
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [23]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

In [24]:
h_pool2_flat= tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

In [25]:
# 드롭아웃
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


In [26]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [27]:
#모델훈련
cress_entropy = -tf.reduce_sum(y_ *tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cress_entropy)

#모델평가
corrent_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(corrent_prediction, "float"))

In [28]:
sess = tf.Session()

In [29]:
sess.run(tf.global_variables_initializer())
for i in range(1000):
    batch = mnist.train.next_batch(100)
    if i%100 == 0 :
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, training accuracy %g" %(i, train_accuracy))
    
    sess.run(train_step, feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    
print("test accuracy %g"% sess.run(accuracy,  feed_dict={x:mnist.test.images, y_:mnist.test.labels
                                                         , keep_prob:1.0}))   

step 0, training accuracy 0.1
step 100, training accuracy 0.91
step 200, training accuracy 0.94
step 300, training accuracy 0.97
step 400, training accuracy 0.86
step 500, training accuracy 0.96
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.94
test accuracy 0.9703
